#### Prevedere se il prezzo di un titolo salirà o scenderà

Da Build A Cryptocurrency Price Movement Classifier

https://www.youtube.com/watch?v=YeYmXNvFrpQ&ab_channel=ComputerScience

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import date, datetime
#plt.style.use('fivethirtyeight')
import yfinance as yf
import pandas as pd

In [2]:
start_date = '2014-05-31'
end_date = datetime.today()
df = yf.download('^GSPC', start_date, end_date) 

[*********************100%%**********************]  1 of 1 completed


In [3]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-05-24,5281.450195,5311.649902,5278.390137,5304.720215,5304.720215,3005510000
2024-05-28,5315.910156,5315.910156,5280.890137,5306.040039,5306.040039,3751540000
2024-05-29,5278.729980,5282.270020,5262.700195,5266.950195,5266.950195,3552750000
2024-05-30,5259.770020,5260.209961,5222.100098,5235.479980,5235.479980,3818750000
2024-05-31,5243.209961,5280.330078,5191.680176,5277.509766,5277.509766,5437160000


Creiamo l'indicatore EMA (Exponential Moving Average)

In [4]:
def EMA(data, period=20, column = 'Close'):
    return data[column].ewm(span=period, adjust=False).mean()

Creiamo l'indicatore RSI (Relative Strength Index)

In [5]:
def RSI(data, period=14, column='Close'):
    delta = data[column].diff(1)
    delta = delta.dropna()
    up = delta.copy()
    down = delta.copy()
    up[up<0] = 0
    down[down>0] = 0
    data['up'] = up
    data['down'] = down
    AVG_Gain = EMA(data, period, column= 'up')
    AVG_Loss = abs(EMA(data, period, column= 'down'))
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    data['RSI' + str(period)] = RSI
    return data

In [6]:
RSI(df, 7)
RSI(df, 14)
RSI(df, 20)
df['EMA15'] = EMA(df,15)
df['EMA20'] = EMA(df,20)
df['EMA50'] = EMA(df,50)

In [7]:
df

,Open,High,Low,Close,Adj Close,Volume,up,down,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50
Date,,,,,,,,,,,,,,
2014-06-02,1923.869995,1925.880005,1915.979980,1924.969971,1924.969971,2509020000,NaN,NaN,NaN,NaN,NaN,1924.969971,1924.969971,1924.969971
2014-06-03,1923.069946,1925.069946,1918.790039,1924.239990,1924.239990,2867180000,0.000000,-0.729980,0.000000,0.000000,0.000000,1924.878723,1924.900449,1924.941344
2014-06-04,1923.060059,1928.630005,1918.599976,1927.880005,1927.880005,2793920000,3.640015,0.000000,62.436399,43.411609,34.421499,1925.253883,1925.184216,1925.056586
2014-06-05,1928.520020,1941.739990,1922.930054,1940.459961,1940.459961,3113270000,12.579956,0.000000,90.311379,79.280231,71.670342,1927.154643,1926.639049,1925.660640
2014-06-06,1942.410034,1949.439941,1942.410034,1949.439941,1949.439941,2864300000,8.979980,0.000000,94.321816,86.387141,80.437204,1929.940305,1928.810562,1926.593161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-24,5281.450195,5311.649902,5278.390137,5304.720215,5304.720215,3005510000,36.880371,0.000000,60.078412,64.919298,64.798724,5252.953796,5233.485835,5161.869328
2024-05-28,5315.910156,5315.910156,5280.890137,5306.040039,5306.040039,3751540000,1.319824,0.000000,60.762917,65.208950,64.990707,5259.589576,5240.395759,5167.523082
2024-05-29,5278.729980,5282.270020,5262.700195,5266.950195,5266.950195,3552750000,0.000000,-39.089844,36.230863,50.858449,55.145416,5260.509653,5242.924753,5171.422184


Creiamo la colonna obiettivo per determinare se il prezzo di domani sarà più alto del prezzo di oggi. Se sì il valore è 1 altrimenti 0

In [8]:
df['Target'] = np.where(df['Close'].shift(-1)> df['Close'], 1, 0)

In [9]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,up,down,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50,Target
Date,,,,,,,,,,,,,,,
2014-06-02,1923.869995,1925.880005,1915.979980,1924.969971,1924.969971,2509020000,NaN,NaN,NaN,NaN,NaN,1924.969971,1924.969971,1924.969971,0
2014-06-03,1923.069946,1925.069946,1918.790039,1924.239990,1924.239990,2867180000,0.000000,-0.72998,0.000000,0.000000,0.000000,1924.878723,1924.900449,1924.941344,1
2014-06-04,1923.060059,1928.630005,1918.599976,1927.880005,1927.880005,2793920000,3.640015,0.00000,62.436399,43.411609,34.421499,1925.253883,1925.184216,1925.056586,1
2014-06-05,1928.520020,1941.739990,1922.930054,1940.459961,1940.459961,3113270000,12.579956,0.00000,90.311379,79.280231,71.670342,1927.154643,1926.639049,1925.660640,1
2014-06-06,1942.410034,1949.439941,1942.410034,1949.439941,1949.439941,2864300000,8.979980,0.00000,94.321816,86.387141,80.437204,1929.940305,1928.810562,1926.593161,1


In [9]:
df.shape

(2518, 15)

Cancelliamo la prima riga di dati

In [10]:
df = df[1:]

In [11]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,up,down,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50,Target
Date,,,,,,,,,,,,,,,
2014-06-03,1923.069946,1925.069946,1918.790039,1924.239990,1924.239990,2867180000,0.000000,-0.72998,0.000000,0.000000,0.000000,1924.878723,1924.900449,1924.941344,1
2014-06-04,1923.060059,1928.630005,1918.599976,1927.880005,1927.880005,2793920000,3.640015,0.00000,62.436399,43.411609,34.421499,1925.253883,1925.184216,1925.056586,1
2014-06-05,1928.520020,1941.739990,1922.930054,1940.459961,1940.459961,3113270000,12.579956,0.00000,90.311379,79.280231,71.670342,1927.154643,1926.639049,1925.660640,1
2014-06-06,1942.410034,1949.439941,1942.410034,1949.439941,1949.439941,2864300000,8.979980,0.00000,94.321816,86.387141,80.437204,1929.940305,1928.810562,1926.593161,1
2014-06-09,1948.969971,1955.550049,1947.160034,1951.270020,1951.270020,2812180000,1.830078,0.00000,94.895908,87.403156,81.711968,1932.606520,1930.949558,1927.560881,0


Facciamo la lista delle colonne da tenere

In [12]:
keep_columns = df.drop(['High', 'Low', 'Open', 'Volume','Adj Close',
                       'up', 'down','Close', 'Target'], axis = 1).columns
df[keep_columns].head()

,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50
Date,,,,,,
2014-06-03,0.000000,0.000000,0.000000,1924.878723,1924.900449,1924.941344
2014-06-04,62.436399,43.411609,34.421499,1925.253883,1925.184216,1925.056586
2014-06-05,90.311379,79.280231,71.670342,1927.154643,1926.639049,1925.660640
2014-06-06,94.321816,86.387141,80.437204,1929.940305,1928.810562,1926.593161
2014-06-09,94.895908,87.403156,81.711968,1932.606520,1930.949558,1927.560881


In [13]:
X = df[keep_columns].values
Y = df['Target'].values

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

Creiamo il modello - Classificatore "Random forest"

L’ Apprendimento ensemble prevedono l’utilizzo di vari modelli d’insieme per migliorare le prestazioni di ognuno di loro preso individualmente. Questi metodi possono essere descritti come tecniche che utilizzano un gruppo di modelli deboli (quelli che in media ottengono risultati solo leggermente migliori di un modello casuale) insieme, al fine di crearne uno più forte e aggregato. Nel nostro caso, le foreste casuali sono un insieme di molti alberi decisionali individuali.

Uno dei principali svantaggi degli alberi decisionali è che sono molto inclini a un adattamento eccessivo: funzionano bene sui dati di addestramento, ma non sono così flessibili per fare previsioni su campioni invisibili. Sebbene ci siano soluzioni alternative per questo, come ridurre gli alberi, questo riduce il loro potere predittivo. Generalmente sono modelli con bias medio e varianza alta, ma sono semplici e di facile interpretazione.

I modelli Random Forest combinano la semplicità degli alberi decisionali com la flessibilità e la potenza di un modello di insieme. In una foresta di alberi, dimentichiamo l’alta varianza di un albero specifico e siamo meno preoccupati per ogni singolo elemento, quindi possiamo coltivare alberi più belli e più grandi che hanno più potere predittivo di uno ridotto. Sebbene i modelli Random Forest non offrano la stessa capacità di interpretazione di un singolo albero, le loro prestazioni sono di gran lunga migliori. 

Utilizzando diversi campioni di dati per addestrare ogni singolo albero riduciamo uno dei problemi principali che hanno: sono molto legati ai loro dati di addestramento. Se addestriamo una foresta con molti alberi e ognuno di loro è stato addestrato con dati diversi, risolviamo questo problema. Sono tutti molto affezionati ai loro dati di allenamento, ma la foresta non ama nessun punto dati specifico. Questo ci consente di coltivare alberi singoli più grandi, poiché non ci interessa più tanto l’overfitting di un singolo albero. Se utilizziamo una porzione molto piccola dell’intero set di dati per addestrare ogni singolo albero, aumentiamo la casualità della foresta (riducendo l’over-fitting) ma di solito a scapito di una prestazione inferiore.

In pratica, per impostazione predefinita, la maggior parte delle implementazioni di Random Forest (come quella di Scikit-Learn) seleziona il campione dei dati di addestramento utilizzato per ogni albero in modo che abbia le stesse dimensioni del set di dati originale (tuttavia non è lo stesso set di dati, ricorda che stiamo selezionando campioni casuali).

https://netai.it/cose-un-modello-random-forest/#page-content

In [15]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=1, criterion='entropy', random_state=1)
forest.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=1, random_state=1)

Verifichiamo la bontà del modello sul data set di test

In [16]:
forest.score(X_test, Y_test)

0.5218253968253969

In [17]:
forest_predictions = forest.predict(X_test)
forest_predictions

array([0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,

In [18]:
Y_test

array([0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,

In [19]:
1 - sum(abs(Y_test - forest_predictions))/len(Y_test)

0.5218253968253967